In [18]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [19]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'tensile_strength'

In [20]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [21]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
fields = ['TensileStrength_PSI_']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

2270


In [22]:
# print(len(df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]))
# df[(df['OutsideDiameter'].astype(str)!='nan')&(df['mm'].astype(str)!='[]')]
df.head(0)

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, keywords, packageweight, packagequantity, shortdescription, Color, CanSize, NetWeight, PackageType, Flammability, PrivateLabel, OperatingTemperature, Size, Type, _ofPieces, Finish, HeadStyle, DriveRecess, Material, Length, Diameter, Application, BandWidth, ScrewSize, UsableLength, OverallLength, GripRange, BodyMaterial, HeadDiameter, MandrelMaterial, HoleSize, ThreadType, ThreadPitch, FlangeDiameter, HeadSize, StemLength, StemSize, IndustryNumber, WidthAcrosstheFlats, OutsideDiameter, ThreadSize, HoleDiameter, Width, Style, Polarity, StudSize, InsideDiameter, Height, TubeSize, BoltSize, PipeSize, ClampRange, HoseO_D_, HoseI_D_, Bore, Scent]
Index: []

In [23]:
df[df['external_id'].astype(str)=='419793']

Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, keywords, packageweight, packagequantity, shortdescription, Color, CanSize, NetWeight, PackageType, Flammability, PrivateLabel, OperatingTemperature, Size, Type, _ofPieces, Finish, HeadStyle, DriveRecess, Material, Length, Diameter, Application, BandWidth, ScrewSize, UsableLength, OverallLength, GripRange, BodyMaterial, HeadDiameter, MandrelMaterial, HoleSize, ThreadType, ThreadPitch, FlangeDiameter, HeadSize, StemLength, StemSize, IndustryNumber, WidthAcrosstheFlats, OutsideDiameter, ThreadSize, HoleDiameter, Width, Style, Polarity, StudSize, InsideDiameter, Height, TubeSize, BoltSize, PipeSize, ClampRange, HoseO_D_, HoseI_D_, Bore, Scent]
Index: []

In [6]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [7]:
# df['MaxPressure'].explode().value_counts()

In [10]:
# # start=r'(?i)(^.{0,10})|()'
# # df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(?i)(\d+\,?\d*\s?P\W?S\W?I\W?)|(psi)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



# pat=r'(?i)(\d+\,?\d*\s?P\W?S\W?I\W?)|()'
# df['matches']=df['TensileStrength_PSI_'].apply(lambda x: re_extract(pat, str(x)))

# # pat=r'(?i)(mm)|()'
# # df['mm']=df['OutsideDiameter'].apply(lambda x: re_extract(pat, str(x)))


# custom=df[(df['TensileStrength_PSI_'].astype(str)!='nan')]
# custom[curation_col]=custom['matches'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'P.?S.?I.?','PSI',str(x)))#.apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
# print(len(custom))
# matchcustom=custom[['external_id',curation_col]]
# # custom[curation_col].explode().value_counts()
# # # custom[custom[curation_col].astype(str)==r'300 150   ']
# custom[0:500]

In [11]:
print(len(df[df['ld'].astype(str)!='[]']))
print(len(df[(df['ld'].astype(str)=='[]')&(df['name'].astype(str)=='[]')]))
na=df[(df['ld'].astype(str)=='[]')&(df['name'].astype(str)=='[]')]

0
2270


In [12]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [14]:
# two=df[(df['matches'].astype(str)=='[]')]
two=df
print(len(two[two['name'].astype(str)!='[]']))
two=two[two['name'].astype(str)!='[]']
two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
two[curation_col].explode().value_counts()
matchtwo=two[['external_id',curation_col]]

0


In [24]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [15]:
# na=df[(df['name'].astype(str)=='[]')&(df['matches'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))

2270


In [16]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

NameError: name 'matchcustom' is not defined

In [11]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [21]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [17]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [47]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 